In [122]:
import torch, math, functools, torch.nn as nn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import pennylane as qml
from pennylane.qnn import TorchLayer

In [123]:
BATCH, STEPS         = 64, 12
FLOW_MIN, FLOW_MAX   = 10, 200
FLOW_RANGE           = FLOW_MAX - FLOW_MIN          # 190
N_IN, N_H            = 3, 6
N_Q                  = N_IN + N_H
EPOCHS               = 40                  # QRNN & MLP
LR_QRNN, LR_MLP      = 8e-3, 8e-3
NOISE_SIGMA          = 0.4                      # 30 % noise


In [124]:
def build_phys_norm(T=STEPS, B=BATCH):
    rng  = torch.Generator().manual_seed(0)
    flow = torch.rand(B, generator=rng)*FLOW_RANGE + FLOW_MIN
    temp = torch.rand(B, generator=rng)*40
    conc = torch.rand(B, generator=rng)*59 + 1
    phys = torch.zeros(T,B,3)
    for t in range(T):
        if t:
            flow = (flow + torch.randn(B)*5).clamp(FLOW_MIN,FLOW_MAX)
            temp = (temp + torch.randn(B)*1.2).clamp(0,40)
            conc = (conc + torch.randn(B)*2.0).clamp(1,60)
        phys[t,:,0] = (flow-FLOW_MIN)/FLOW_RANGE
        phys[t,:,1] = temp/40
        phys[t,:,2] = (conc-1)/59
    return phys

In [125]:
def raw_eff(zf, zt, zc):
    f = FLOW_MIN + zf*FLOW_RANGE
    t = zt*40
    c = zc*59 + 1
    mu_f  = 80 + 0.4*(t-20)
    sigma = 3000 + 30*(c-30).abs()
    return 1 \
         - 0.35*((f-mu_f)**2)/sigma \
         - 0.25*((t-20)/20)**2 \
         - 0.15*((c-30)/29.5)**2

def optimal_flow(z_t, z_c, n_grid=401):
    z_grid = torch.linspace(0,1,n_grid)
    eff = raw_eff(z_grid.unsqueeze(1), z_t.unsqueeze(0), z_c.unsqueeze(0))
    return z_grid[eff.argmax(dim=0)]

def qubit(z):
    θ = z*math.pi
    return torch.stack([torch.cos(θ/2), torch.sin(θ/2)])

def encode_to_amp(zf,zt,zc):
    vec = functools.reduce(torch.kron,
        [qubit(z) for z in (zf,zt,zc)] + [torch.tensor([1.,0.])]*N_H)
    return vec/vec.norm()

phys = build_phys_norm()
X_amp = torch.stack([
    torch.stack([encode_to_amp(*phys[t,b]) for b in range(BATCH)])
    for t in range(STEPS)
])                                              # (T,B,128)
last   = phys[-1]
Y_true = optimal_flow(last[:,1], last[:,2]).unsqueeze(1)   # (B,1)

# ╔══════════════════════ 2 ─ QRNN  ════════════════════════════════════════╗
dev_rec = qml.device("default.qubit", wires=N_Q)
dev_out = qml.device("default.qubit", wires=N_H)


In [126]:
@qml.qnode(dev_rec, interface="torch")
def qrnn_cell(inputs, weights):
    qml.AmplitudeEmbedding(inputs, wires=range(N_Q), normalize=True)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(N_Q))
    return [qml.expval(qml.PauliZ(i)) for i in range(N_IN,N_Q)]

qrnn_layer = TorchLayer(qrnn_cell, {"weights": (3,N_Q,3)}); qrnn_layer.batch_execute=False

@qml.qnode(dev_out, interface="torch")
def readout(inputs, weights):
    for j in range(N_H):
        qml.RY(inputs[j], wires=j)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(N_H))
    return qml.expval(qml.PauliZ(0))

read_layer = TorchLayer(readout, {"weights": (4,N_H,3)}); read_layer.batch_execute=False

class QRNN(nn.Module):
    def __init__(self):
        super().__init__(); self.q=qrnn_layer; self.r=read_layer
    def forward(self,x):
        B=x.shape[1]; h=torch.zeros(B,N_H)
        for t in range(x.shape[0]):
            h=torch.stack([self.q(x[t,b]) for b in range(B)])
        return torch.stack([self.r(h[b]) for b in range(B)]).unsqueeze(1)

model_q = QRNN(); opt_q=torch.optim.Adam(model_q.parameters(), lr=LR_QRNN)
loss_fn = nn.MSELoss()

In [ ]:
print("\n⚡ Training QRNN (50 epochs) – first 3 samples each epoch")
for ep in range(1, EPOCHS+1):
    opt_q.zero_grad()
    pred_z = model_q(X_amp)
    loss   = loss_fn(pred_z, Y_true)
    loss.backward(); opt_q.step()
    tf = Y_true.squeeze()*FLOW_RANGE + FLOW_MIN
    pf = pred_z.squeeze()*FLOW_RANGE + FLOW_MIN
    print(f"Epoch {ep:2d} | MSE={loss.item():.4f} | sample0 true={tf[0]:.1f}  pred={pf[0]:.1f}")

# ╔══════════════════════ 3 ─ BAD BASELINE & SMALL MLP ═════════════════════╗
baseline_pred = torch.full_like(Y_true, (100-FLOW_MIN)/FLOW_RANGE)   # very poor guess

class TinyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3,8), nn.ReLU(),
            nn.Linear(8,4), nn.ReLU(),
            nn.Linear(4,1)
        )
    def forward(self,x): return self.net(x)

# Poisson noise replacement
lambda_param = last * 20  # Scale inputs to count-based intensities
noisy_counts = torch.poisson(lambda_param)
noisy_inputs = (noisy_counts / 20).clamp(0, 1)
mlp = TinyMLP(); opt_m=torch.optim.Adam(mlp.parameters(), lr=LR_MLP)
for ep in range(1, EPOCHS+1):
    opt_m.zero_grad(); p=mlp(noisy_inputs); l=loss_fn(p, Y_true)
    l.backward(); opt_m.step()


⚡ Training QRNN (50 epochs) – first 3 samples each epoch
Epoch  1 | MSE=0.1533 | sample0 true=85.5  pred=4.8
Epoch  2 | MSE=0.1331 | sample0 true=85.5  pred=9.9
Epoch  3 | MSE=0.1138 | sample0 true=85.5  pred=15.2
Epoch  4 | MSE=0.0955 | sample0 true=85.5  pred=20.7
Epoch  5 | MSE=0.0786 | sample0 true=85.5  pred=26.3
Epoch  6 | MSE=0.0632 | sample0 true=85.5  pred=31.9
Epoch  7 | MSE=0.0495 | sample0 true=85.5  pred=37.5
Epoch  8 | MSE=0.0377 | sample0 true=85.5  pred=43.0
Epoch  9 | MSE=0.0278 | sample0 true=85.5  pred=48.4
Epoch 10 | MSE=0.0197 | sample0 true=85.5  pred=53.6
Epoch 11 | MSE=0.0133 | sample0 true=85.5  pred=58.6
Epoch 12 | MSE=0.0084 | sample0 true=85.5  pred=63.4
Epoch 13 | MSE=0.0049 | sample0 true=85.5  pred=67.8
Epoch 14 | MSE=0.0026 | sample0 true=85.5  pred=71.9
Epoch 15 | MSE=0.0013 | sample0 true=85.5  pred=75.6
Epoch 16 | MSE=0.0007 | sample0 true=85.5  pred=79.0
Epoch 17 | MSE=0.0007 | sample0 true=85.5  pred=82.1
Epoch 18 | MSE=0.0011 | sample0 true=85.5  

In [ ]:
with torch.no_grad():
    pred_q  = model_q(X_amp).squeeze()
    pred_m  = mlp(noisy_inputs).squeeze()
    true_z  = Y_true.squeeze()

# ╔══════════════════════ 4 ─ 3-D SCATTER ══════════════════════════════════╗
fig = plt.figure(figsize=(8,6))
ax  = fig.add_subplot(111, projection='3d')

temps = last[:,1]*40
concs = last[:,2]*59 + 1
f_true = true_z*FLOW_RANGE + FLOW_MIN
f_q    = pred_q*FLOW_RANGE + FLOW_MIN
f_m    = pred_m*FLOW_RANGE + FLOW_MIN
f_b    = baseline_pred.squeeze()*FLOW_RANGE + FLOW_MIN

ax.scatter(temps, concs, f_true, c='k', marker='o', label='True optimum')
ax.scatter(temps, concs, f_q,    c='g', marker='^', label='QRNN')
ax.scatter(temps, concs, f_m,    c='r', marker='s', label='Tiny MLP noisy')
ax.scatter(temps, concs, f_b,    c='b', marker='x', label='Baseline 150 m³/s')
ax.set_xlabel("Temp (°C)"); ax.set_ylabel("Conc (PSU)"); ax.set_zlabel("Flow (m³/s)")
ax.set_title("Optimal flow prediction – degraded baselines"); ax.legend()
plt.tight_layout(); plt.show()

# ╔══════════════════════ 5 ─ MEAN-EFFICIENCY BAR CHART ════════════════════╗
eff_true = raw_eff(true_z, last[:,1], last[:,2])
eff_q    = raw_eff(pred_q, last[:,1], last[:,2])
eff_b    = raw_eff(baseline_pred.squeeze(), last[:,1], last[:,2])
eff_m    = raw_eff(pred_m, last[:,1], last[:,2])

means = [eff_q.mean().item(), eff_b.mean().item(), eff_m.mean().item()]
labels= ["QRNN", "Baseline", "Tiny MLP"]
colors= ["green", "blue", "red"]

plt.figure(figsize=(6,4))
bars = plt.bar(labels, means, color=colors, edgecolor="k")
plt.axhline(eff_true.mean().item(), ls="--", color="k", label="True optimum")
for bar,val in zip(bars, means):
    plt.text(bar.get_x()+bar.get_width()/2, val+0.01, f"{val:.3f}",
             ha="center", va="bottom")
plt.ylim(0,1)
plt.ylabel("Mean efficiency")
plt.title("Average efficiency – QRNN vs degraded baselines")
plt.legend(); plt.tight_layout(); plt.show()

# ╔══════════════════════ 6 ─ MSE SUMMARY ══════════════════════════════════╗
mse_q = loss_fn(pred_q,              true_z).item()
mse_b = loss_fn(baseline_pred.squeeze(), true_z).item()
mse_m = loss_fn(pred_m,              true_z).item()
print(f"\nMSE QRNN     : {mse_q:.4f}")
print(f"MSE Baseline : {mse_b:.4f}")
print(f"MSE Tiny MLP : {mse_m:.4f}")